### Preparando o ambiente

In [ ]:
# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ciclo11_Python_Para_Dados/mod3/

/content/drive/MyDrive/ciclo11_Python_Para_Dados/mod3


In [ ]:
import os
print(os.getcwd())


/content/drive/MyDrive/ciclo11_Python_Para_Dados/mod3


In [ ]:
!ls

mod3_2.json  Untitled0.ipynb  vendas_carros.db


# Aula Prática - Consultas Avançadas em SQL com Python e SQLite

## Objetivo da Aula

Nesta aula prática, vamos explorar consultas avançadas em SQL, como:
    - Filtros com `WHERE`

    - Agrupamento de resultados com `GROUP BY`

    - Filtragem de grupos com `HAVING`

    - Funções de agregação (ex: `COUNT`, `SUM`, `AVG`)

    - Junção de tabelas com `JOIN`

    Usaremos Python e a biblioteca SQLite para realizar essas consultas em um banco de dados de exemplo.

## Configuração Inicial: Conectar ao Banco de Dados

In [ ]:
import sqlite3

In [ ]:
# Conectar ao banco de dados (ou criar, se não existir)
conexao = sqlite3.connect('empresa.db')
cursor = conexao.cursor()

In [ ]:
# Criar tabelas de exemplo para a aula
cursor.execute('''
CREATE TABLE IF NOT EXISTS Funcionarios (
    id_funcionario INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    cargo TEXT NOT NULL,
    salario REAL NOT NULL
)
''')

In [ ]:

cursor.execute('''
CREATE TABLE IF NOT EXISTS Departamentos (
    id_departamento INTEGER PRIMARY KEY AUTOINCREMENT,
    nome_departamento TEXT NOT NULL
)
''')

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Alocacoes (
    id_alocacao INTEGER PRIMARY KEY AUTOINCREMENT,
    id_funcionario INTEGER,
    id_departamento INTEGER,
    FOREIGN KEY (id_funcionario) REFERENCES Funcionarios(id_funcionario),
    FOREIGN KEY (id_departamento) REFERENCES Departamentos(id_departamento)
)
''')


In [ ]:

# Inserindo dados de exemplo
funcionarios = [
    ('Alice', 'Engenheira de Software', 7000),
    ('Bob', 'Gerente de Projetos', 8500),
    ('Carlos', 'Analista de Dados', 6000),
    ('Diana', 'Engenheira de Software', 7200)
]

In [ ]:
departamentos = [
    ('Desenvolvimento',),
    ('Dados',),
    ('Gestão de Projetos',)
]

In [ ]:
alocacoes = [
    (1, 1),  # Alice -> Desenvolvimento
    (2, 3),  # Bob -> Gestão de Projetos
    (3, 2),  # Carlos -> Dados
    (4, 1)   # Diana -> Desenvolvimento
]


In [ ]:
cursor.executemany("INSERT INTO Funcionarios (nome, cargo, salario) VALUES (?, ?, ?)", funcionarios)
cursor.executemany("INSERT INTO Departamentos (nome_departamento) VALUES (?)", departamentos)
cursor.executemany("INSERT INTO Alocacoes (id_funcionario, id_departamento) VALUES (?, ?)", alocacoes)

In [ ]:

# Salvar alterações
conexao.commit()

## Consulta Avançada 1: Junção de Tabelas com `JOIN`"

In [ ]:
# Consultar dados de funcionários e seus departamentos
cursor.execute('''
SELECT Funcionarios.nome, Funcionarios.cargo, Departamentos.nome_departamento
FROM Funcionarios
JOIN Alocacoes ON Funcionarios.id_funcionario = Alocacoes.id_funcionario
JOIN Departamentos ON Alocacoes.id_departamento = Departamentos.id_departamento
''')

In [ ]:
# Exibir resultados
resultados = cursor.fetchall()
for resultado in resultados:
    print(f"Cargo: {resultado[0]}, Salário Médio: {resultado[1]}")


## Consulta Avançada 2: Funções de Agregação e `GROUP BY

In [ ]:
# Consulta: Salário médio por cargo
cursor.execute('''
SELECT cargo, AVG(salario) AS salario_medio
FROM Funcionarios
GROUP BY cargo
''')


In [ ]:
# Exibir resultados
resultados = cursor.fetchall()
for resultado in resultados:
    print(f"Cargo: {resultado[0]}, Salário Médio: {resultado[1]}")



## Consulta Avançada 3: Filtragem com `HAVING`

In [ ]:
# Consulta: Departamentos com mais de 1 funcionário
cursor.execute('''
SELECT Departamentos.nome_departamento, COUNT(Funcionarios.id_funcionario) AS total_funcionarios
FROM Funcionarios
JOIN Alocacoes ON Funcionarios.id_funcionario = Alocacoes.id_funcionario
JOIN Departamentos ON Alocacoes.id_departamento = Departamentos.id_departamento
GROUP BY Departamentos.nome_departamento
HAVING COUNT(Funcionarios.id_funcionario) > 1
''')

In [ ]:
# Exibir resultados
resultados = cursor.fetchall()
for resultado in resultados:
    print(f"Departamento: {resultado[0]}, Total de Funcionários: {resultado[1]}")